# Sum Spots

https://www.reddit.com/r/puzzles/comments/1men71x/sum_spotslevel_of_difficulty_high/

u/mhmhbetter1 posted a picture of a puzzle from a Penny Dell pulp puzzle magazine. This one seems to be a Magic Square variant, probably easier.

It's always fun to practice constrain programming with Z3.

## Problem Transcription

In [ ]:
n = 4

In [ ]:
# (value, [upper-left dot, upper-right dot, lower-left dot, lower-right dot])
tiles = [
    (28, [True, False, False, True]),
    (22, [True, False, False, True]),
    (8, [True, False, True, True]),
    (19, [True, False, True, True]),
    (12, [True, False, False, True]),
    (17, [True, False, False, True]),
    (21, [True, True, True, False]),
    (5, [False, True, True, False]),
    (23, [False, True, True, True]),
    (9, [False, True, True, False]),
    (10, [True, True, False, True]),
    (6, [True, False, False, True]),
    (18, [False, True, True, False]),
    (24, [False, True, True, True]),
    (15, [True, True, True, False]),
    (7, [False, True, True, False]),
]

# (index, (row, col))
placed_tiles = [(n*n-1, (2,2))]

## Problem Definition in Z3

We can define the problem by making an variable for each cell in the grid, these values must be distinct, and can only take on the values of the tiles. For a given tile value will imply the dot pattern around it. Notice by transitivity all dots around a corner must have the same color (even though the instructions say this isn't necessarily so), so we can define a single boolean variable for the dot color at every shared corner.

In [ ]:
import z3

### Variables

As state, we need an integer variable for every cell in the grid.

In [ ]:
grid_vars = []
for i in range(n):
    grid_vars.append([])
    for j in range(n):
        grid_vars[-1].append(z3.Int(f"v_{i}_{j}"))

grid_vars

And a boolean variable for every shared corner.

In [ ]:
dot_vars = []
for i in range(n+1):
    dot_vars.append([])
    for j in range(n+1):
        dot_vars[-1].append(z3.Bool(f"d_{i}_{j}"))
        
dot_vars

### Constraints

In [ ]:
solver = z3.Solver()

We can add the initial conditions by simple specifying the cell value.

In [ ]:
for index,(i,j) in placed_tiles:
    v,dots = tiles[index]
    solver.add(grid_vars[i][j] == v)

We should make sure the each cell's value must be one of the tiles. Also given a cell's value, it implies a specific dot pattern about it.

In [ ]:
for i,row in enumerate(grid_vars):
    for j,var in enumerate(row):
        # must be one of the tiles
        solver.add(z3.Or([var == v for v,dots in tiles]))
        # each tiles has its dot pattern
        for v,dots in tiles:
            solver.add(z3.Implies(var == v, z3.And([dot_vars[i + k//2][j + k%2] == dots[k] for k in range(4)])))

The cells must be unique values.

In [ ]:
solver.add(z3.Distinct([var for row in grid_vars for var in row]))

Finally, we have the constraint that every row and column, and diagonal, must sum to 61

In [ ]:
for i in range(n):
    solver.add(sum(grid_vars[i][j] for j in range(n)) == 61)
    solver.add(sum(grid_vars[j][i] for j in range(n)) == 61)
solver.add(sum(grid_vars[i][i] for i in range(n)) == 61)
solver.add(sum(grid_vars[n-1-i][i] for i in range(n)) == 61)

In [ ]:
solver.assertions()

## Performing the Solve

In [ ]:
%%time
solver.check()

## Checking Results

In [ ]:
model = solver.model()

In [ ]:
solution = [[model.evaluate(var).as_long() for j,var in enumerate(row)] for i,row in enumerate(grid_vars)]

for row in solution:
    print(" ".join(list(map(str,row))))